# 계절성 지수 산출 프로그램

In [85]:
import pandas as pd

In [86]:
# 설정을 기본값으로 되돌리기
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

# 모든 행과 열을 값으로 표시하기 위해 display.max_rows 및 display.max_columns 설정
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [87]:
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
# 1.23456789e-10 -> 0.000000000123456789
pd.reset_option('display.float_format')

In [88]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [89]:
selloutData = pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")

In [90]:
selloutData["REGIONID"] = selloutData["REGIONID"].astype(str)
selloutData["PRODUCT"] = selloutData["PRODUCT"].astype(str)
selloutData["YEARWEEK"] = selloutData["YEARWEEK"].astype(str)
selloutData["QTY"] = selloutData["QTY"].astype(float)

In [91]:
def qtyRefine(inValue):
    # inValue=-5
    
    outValue = 0
    if inValue < 0:
       outValue = 0
    else:
        outValue = inValue
    return outValue

In [92]:
selloutData["QTY_NEW"] = selloutData.QTY.apply(qtyRefine)

In [94]:
selloutData["YEAR"] = selloutData.YEARWEEK.str[:4]
selloutData["WEEK"] = selloutData.YEARWEEK.str[4:]

In [116]:
refinedSelloutData = selloutData.loc[selloutData.WEEK != "53"]

In [118]:
sortKey = ["REGIONID","PRODUCT","YEARWEEK"]

In [119]:
sortedData = refinedSelloutData.sort_values(sortKey)

In [121]:
groupKey = ["REGIONID","PRODUCT","YEAR"]

In [126]:
groupData = sortedData.groupby(groupKey)[["QTY_NEW"]].agg("mean").reset_index()

In [128]:
groupData.rename(columns={"QTY_NEW":"QTY_MEAN"}, inplace=True)

In [130]:
groupKey = ["REGIONID","PRODUCT","YEAR"]

In [132]:
mergedData = pd.merge(left=refinedSelloutData,
         right=groupData,
         on=groupKey)

In [135]:
mergedData["SEASONALITY"] = mergedData.QTY_NEW / mergedData.QTY_MEAN

In [136]:
groupKey = ["REGIONID","PRODUCT","WEEK"]

In [140]:
finalResult = mergedData.groupby(by=groupKey)[["SEASONALITY"]].agg("mean").reset_index()